In [ ]:
# access folder up
import sys
import os
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from TS_functions import create_sliding_windows
from strategies import RECMO, DIRMO, DIRREC, STRATIFY, FixedEnsemble, Stratify
from forecasting_functions import torch_simple_MLP
from tqdm import tqdm
import numpy as np


dataset_name = 'mg_10000'
train_p, val_p, test_p = 0.2, 0.1, 0.1
window_size = 100
H_ahead = 20

univariate_time_series = np.load(f'univariate_time_series/{dataset_name}.npy')
input_windows, output_windows = create_sliding_windows(univariate_time_series, window_size, H_ahead)
train_N, val_N, test_N = int(train_p*len(input_windows)), int(val_p*len(input_windows)), int(test_p*len(input_windows))
forc_xs, forc_ys = input_windows[:train_N], output_windows[:train_N]
val_xs, val_ys = input_windows[train_N:train_N+val_N], output_windows[train_N:train_N+val_N]

model_save_location = 'example_run_models/'
# make the directory if it doesnt already exist
os.makedirs(model_save_location, exist_ok=True)
# print location
print(f"Model save location: {model_save_location}")
print('warning, if you want to run different experiment settings, add these to the save folder path otherwise you will load the wrong models.')

In [ ]:
xs, ys = forc_xs, forc_ys
base_forecaster = RECMO(torch_simple_MLP(hidden_size=100, epochs = 20), H_ahead, s_parameter = 1)
residual_forecaster = RECMO(torch_simple_MLP(hidden_size=100, epochs = 20), H_ahead, s_parameter = 2)
rectifier = RECMO(torch_simple_MLP(hidden_size=100, epochs = 200), 2, s_parameter = 2)

stratify_recmo = Stratify(base_forecaster, residual_forecaster, rectifier, H_ahead)
stratify_recmo.fit(xs, ys, save_location = model_save_location)
base_fc = stratify_recmo.get_base_forecast(xs)
residual_fc = stratify_recmo.get_residual_forecast(xs)
rectified_fc = stratify_recmo.get_rectified_forecast(xs)

# print errors
base_fc_errors = np.mean((base_fc - ys)**2)
residual_fc_errors = np.mean((residual_fc - ys)**2)
rectified_fc_errors = np.mean((rectified_fc - ys)**2)

print("Base Forecast Errors:", base_fc_errors)
print("Residual Forecast Errors:", residual_fc_errors)
print("Rectified Forecast Errors:", rectified_fc_errors)

In [ ]:

xs, ys = forc_xs, forc_ys
base_forecaster = RECMO(torch_simple_MLP(hidden_size=100, epochs = 20), H_ahead, s_parameter = 1)
residual_forecaster = DIRMO(torch_simple_MLP(hidden_size=100, epochs = 20), H_ahead, s_parameter = 2)

stratify_dirmo = Stratify(base_forecaster, residual_forecaster, deepcopy(residual_forecaster), H_ahead)
stratify_dirmo.fit(xs, ys, save_location = model_save_location)
base_fc = stratify_dirmo.get_base_forecast(xs)
residual_fc = stratify_dirmo.get_residual_forecast(xs)
# print errors
base_fc_errors = np.mean((base_fc - ys)**2)
residual_fc_errors = np.mean((residual_fc - ys)**2)
print("Base Forecast Errors:", base_fc_errors)
print("Residual Forecast Errors:", residual_fc_errors)

In [ ]:
xs, ys = forc_xs, forc_ys
base_forecaster = RECMO(torch_simple_MLP(hidden_size=100, epochs = 20), H_ahead, s_parameter = 1)
residual_forecaster = DIRREC(torch_simple_MLP(hidden_size=100, epochs = 20), H_ahead, s_parameter = 2)
rectifier = DIRREC(torch_simple_MLP(hidden_size=100, epochs = 200), 2, s_parameter = 2)

stratify_dirrec = Stratify(base_forecaster, residual_forecaster, rectifier, H_ahead)
stratify_dirrec.fit(xs, ys, save_location = model_save_location)
base_fc = stratify_dirrec.get_base_forecast(xs)
residual_fc = stratify_dirrec.get_residual_forecast(xs)
rectified_fc = stratify_dirrec.get_rectified_forecast(xs)
# print errors
base_fc_errors = np.mean((base_fc - ys)**2)
residual_fc_errors = np.mean((residual_fc - ys)**2)
rectified_fc_errors = np.mean((rectified_fc - ys)**2)
print("Base Forecast Errors:", base_fc_errors)
print("Residual Forecast Errors:", residual_fc_errors)
print("Rectified Forecast Errors:", rectified_fc_errors)


In [ ]:
recmo_stratify_train_fc = stratify_recmo.predict(forc_xs, decompose = True)
dirmo_stratify_train_fc = stratify_dirmo.predict(forc_xs, decompose = True)
dirrec_stratify_train_fc = stratify_dirrec.predict(forc_xs, decompose = True)
recmo_stratify_val_fc = stratify_recmo.predict(val_xs, decompose = True)
dirmo_stratify_val_fc = stratify_dirmo.predict(val_xs, decompose = True)
dirrec_stratify_val_fc = stratify_dirrec.predict(val_xs, decompose = True)

In [ ]:
import pandas as pd
def get_horizon_level_stats(strategy_errors):
    stats_df = pd.DataFrame()
    stats_df['mean'] = strategy_errors.mean(axis = 0)
    stats_df['std'] = strategy_errors.std(axis = 0)
    stats_df['min'] = strategy_errors.min(axis = 0)
    stats_df['max'] = strategy_errors.max(axis = 0)
    stats_df['median'] = np.median(strategy_errors,axis = 0)
    stats_df['q1'] = np.quantile(strategy_errors, 0.25, axis = 0)
    stats_df['q3'] = np.quantile(strategy_errors, 0.75, axis = 0)
    return stats_df

# forecasts in the dictionaru
forecasts = [recmo_stratify_train_fc['base'], recmo_stratify_train_fc['rectified'], recmo_stratify_train_fc['residual']]
forecast_names = ['Base', 'Rectified', 'Residual']
stats = [get_horizon_level_stats((forecast - ys)**2) for forecast in forecasts]

for idx, stat in enumerate(stats):
    x = np.arange(1, H_ahead + 1)
    y_mean = stat['mean']
    y_std = stat['std']
    y_min = stat['min']
    y_max = stat['max']
    y_median = stat['median']
    y_q1 = stat['q1']
    y_q3 = stat['q3']
    
    plt.figure(figsize=(8, 6))
    plt.fill_between(x, y_min, y_max, alpha=0.2, label='Min–Max range')
    plt.fill_between(x, y_q1, y_q3, alpha=0.4, label='IQR (25–75)')
    plt.plot(x, y_mean, marker='o', linestyle='-', label='Mean')
    # plt.yscale('log')
    plt.xlabel('Time delta')
    plt.ylabel('MSE metric (lower is better)')
    plt.title(f'MSE vs time delta for {forecast_names[idx]}')
    plt.legend()
    plt.show()

In [ ]:
# Create an array for the x-axis (time deltas)
x = np.arange(1, H_ahead + 1)
for stat_name in stats[0].columns:
    plt.figure(figsize=(8, 6))
    for idx, stat in enumerate(stats):
        plt.plot(x, stat[stat_name], marker='o', linestyle='-', label=f'{forecast_names[idx]}')
    plt.xlabel('Time delta')
    plt.ylabel('MSE metric (lower is better)')
    plt.title(f'{stat_name} vs Time Delta for all forecasts')
    plt.legend()
    plt.show()

In [ ]:
val_forecasts = [recmo_stratify_val_fc['base'], recmo_stratify_val_fc['rectified'], recmo_stratify_val_fc['residual']]
val_forecast_names = ['Base', 'Rectified', 'Residual']
val_stats = [get_horizon_level_stats((forecast - val_ys)**2) for forecast in val_forecasts]

for idx, stat in enumerate(val_stats):
    x = np.arange(1, H_ahead + 1)
    y_mean = stat['mean']
    y_std = stat['std']
    y_min = stat['min']
    y_max = stat['max']
    y_median = stat['median']
    y_q1 = stat['q1']
    y_q3 = stat['q3']
    
    plt.figure(figsize=(8, 6))
    plt.fill_between(x, y_min, y_max, alpha=0.2, label='Min–Max range')
    plt.fill_between(x, y_q1, y_q3, alpha=0.4, label='IQR (25–75)')
    plt.plot(x, y_mean, marker='o', linestyle='-', label='Mean')
    # plt.yscale('log')
    plt.xlabel('Time delta')
    plt.ylabel('MSE metric (lower is better)')
    plt.title(f'MSE vs time delta for {forecast_names[idx]} on validation set')
    plt.legend()
    plt.show()

In [ ]:
for stat_name in val_stats[0].columns:
    plt.figure(figsize=(8, 6))
    for idx, stat in enumerate(val_stats):
        plt.plot(x, stat[stat_name], marker='o', linestyle='-', label=f'{forecast_names[idx]}')
    plt.xlabel('Time delta')
    plt.ylabel('MSE metric (lower is better)')
    plt.title(f'{stat_name} vs Time Delta for all forecasts on validation set')
    plt.legend()
    plt.show()

In [ ]:
val_forecasts = [recmo_stratify_val_fc['base'], recmo_stratify_val_fc['rectified'], recmo_stratify_val_fc['residual'],
                    dirmo_stratify_val_fc['base'] , dirmo_stratify_val_fc['residual'],
                    dirrec_stratify_val_fc['base'], dirrec_stratify_val_fc['rectified'], dirrec_stratify_val_fc['residual']]
val_forecast_names = ['Base_recmo', 'Rectified_recmo', 'Residual_recmo',
                        'Base_dirmo', 'Residual_dirmo',
                        'Base_dirrec', 'Rectified_dirrec', 'Residual_dirrec']
val_stats = [get_horizon_level_stats((forecast - val_ys)**2) for forecast in val_forecasts]

for idx, stat in enumerate(val_stats):
    x = np.arange(1, H_ahead + 1)
    y_mean = stat['mean']
    y_std = stat['std']
    y_min = stat['min']
    y_max = stat['max']
    y_median = stat['median']
    y_q1 = stat['q1']
    y_q3 = stat['q3']
    
    plt.figure(figsize=(8, 6))
    plt.fill_between(x, y_min, y_max, alpha=0.2, label='Min–Max range')
    plt.fill_between(x, y_q1, y_q3, alpha=0.4, label='IQR (25–75)')
    plt.plot(x, y_mean, marker='o', linestyle='-', label='Mean')
    # plt.yscale('log')
    plt.xlabel('Time delta')
    plt.ylabel('MSE metric (lower is better)')
    plt.title(f'MSE vs time delta for {val_forecast_names[idx]} on validation set')
    plt.legend()
    plt.show()

In [ ]:
for stat_name in val_stats[0].columns:
    plt.figure(figsize=(8, 6))
    for idx, stat in enumerate(val_stats):
        plt.plot(x, stat[stat_name], marker='o', linestyle='-', label=f'{val_forecast_names[idx]}')
    plt.xlabel('Time delta')
    plt.ylabel('MSE metric (lower is better)')
    plt.title(f'{stat_name} vs Time Delta for all forecasts on validation set')
    plt.legend()
    plt.yscale('log')
    plt.show()